In [10]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
# データセットのロード
# iris.data = [(がく片の長さ , がく片の幅 , 花びらの長さ , 花びらの幅)]
iris = datasets.load_iris()
x_vals = np.array([x[0:3] for x in iris.data])
y_vals = np.array([x[3] for x in iris.data])
sess = tf.Session()
seed = 2
tf.set_random_seed(seed)
np.random.seed(seed)

#  ８割のデータをトレーニングセットに設定
train_indices = np.random.choice(len(x_vals), round(len(x_vals)*0.8), replace=False)
# 残りのデータをテストセットに設定
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))

x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]

# 正規化
def normalize_cols(m):
    col_max = m.max(axis=0)
    col_min = m.min(axis=0)
    return (m-col_min) / (col_max - col_min)
    
x_vals_train = np.nan_to_num(normalize_cols(x_vals_train))
x_vals_test = np.nan_to_num(normalize_cols(x_vals_test))
batch_size = 30
x_data = tf.placeholder(shape=[None, 3], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
hidden_layer_nodes = 10
W1 = tf.Variable(tf.random_normal(shape=[3,hidden_layer_nodes])) 
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))  
W2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,1])) 
b2 = tf.Variable(tf.random_normal(shape=[1]))  


# 各層の出力定義
hidden_output = tf.nn.relu(tf.add(tf.matmul(x_data, W1), b1))
final_output = tf.nn.relu(tf.add(tf.matmul(hidden_output, W2), b2))

# 誤差関数
loss = tf.reduce_mean(tf.square(y_target - final_output))

# 勾配降下最適化アルゴリズム
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)
# 試してみよう
# train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
# train_step = tf.train.AdagradOptimizer(1e-4).minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

# 学習ループ
loss_vec = []
test_loss = []
epoch = 1000 #学習回数
for i in range(epoch):
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(np.sqrt(temp_loss))
    
    test_temp_loss = sess.run(loss, feed_dict={x_data: x_vals_test, y_target: np.transpose([y_vals_test])})
    test_loss.append(np.sqrt(test_temp_loss))
    if (i+1)%50==0:
        print('Generation: ' + str(i+1) + '. 誤差 = ' + str(temp_loss))

Generation: 50. 誤差 = 1.9669999
Generation: 100. 誤差 = 1.9516667
Generation: 150. 誤差 = 1.507
Generation: 200. 誤差 = 1.627
Generation: 250. 誤差 = 2.1773334
Generation: 300. 誤差 = 1.9156667
Generation: 350. 誤差 = 1.7163333
Generation: 400. 誤差 = 2.1253333
Generation: 450. 誤差 = 2.3933332
Generation: 500. 誤差 = 2.9623332
Generation: 550. 誤差 = 2.0166667
Generation: 600. 誤差 = 2.095
Generation: 650. 誤差 = 1.7633332
Generation: 700. 誤差 = 1.8019999
Generation: 750. 誤差 = 2.2559998
Generation: 800. 誤差 = 1.5076665
Generation: 850. 誤差 = 2.1916666
Generation: 900. 誤差 = 2.1936667
Generation: 950. 誤差 = 2.0933335
Generation: 1000. 誤差 = 1.8933332
